<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [10]</a>'.</span>

In [1]:
from IPython.display import Code

# Example: Comparison of RISC-V SW Toolchains with MLonMCU

TODO

## Supported components

**Models:** Any (`toycar` used below)

**Frontends:** Any (`tflite` used below)

**Frameworks/Backends:** Any (`tvmaotplus` used below)

**Platforms/Targets:** `etiss`, `spike`, `ovpsim`,... (`etiss` used below)

## Prerequisites

If not done already, setup a virtual python environment and install the required packages into it. (See `requirements.txt`)

In [2]:
Code(filename="requirements.txt")

-r ../../requirements.txt

Set up MLonmCU as usual, i.e. initialize an environment and install all required dependencies. Feel free to use the following minimal `environment.yml.j2` template:

In [3]:
Code(filename="environment.yml.j2")

---
# The MLONMCU_HOME is filled in automatically when creating the environment
home: "{{ home_dir }}"
logging:
  level: INFO
  to_file: false
  rotate: false
cleanup:
  auto: true
  keep: 5
# Default locations for certain directoriescan be changed here
# Non-absolute paths will always be threated relative to the MLONMCU_HOME
paths:
  # Where the dependencies are downloaded and installed
  deps: deps
  # If logging to file is used keep logs in this directory
  logs: logs
  # Location where reports and artifacts are written to
  results: results
  # Directory where custom extensions can be integrated
  plugins: plugins
  # Directory for intermediate build products, should be located on a large enough drive
  temp: temp
  # A collection of models which will be used to look for models
  # The paths will be checked in the order defined here stopping at the first match
  # Non-existant paths will be skipped without throwing an error
  models:
    - "{{ home_dir }}/models"
# Here default clone_urls
repos:
  tvm:
    url: "https://github.com/apache/tvm.git"
    ref: v0.17.0
    options:
      recursive: true
  mlif:
    url: "https://github.com/tum-ei-eda/mlonmcu-sw.git"
    ref: c7b0e7364c232666ebd5a17476181e3ff9a71d70
  etiss:
    url: "https://github.com/tum-ei-eda/etiss.git"
    ref: 739573d5f12798074bc7d375a46c005f23f59f67  # develop, 23.01.2024
frameworks:
  default: tvm
  tvm:
    enabled: true
    backends:
      default: tvmaotplus
      tvmaotplus:
        enabled: true
        features: {}
    features: {}
frontends:
  tflite:
    enabled: true
    features: []
toolchains:
  gcc: true
  llvm: true
platforms:
  mlif:
    enabled: true
    features:
      debug: true
      vext: true
      auto_vectorize: true
targets:
  default: host_x86
  host_x86:
    enabled: true
    features: []
  etiss:
    enabled: true
    features:
      trace: true
      log_instrs: true
postprocesses:
  use: []
vars:
  allow_extensions: false
  runs_per_stage: true
  llvm.dl_url: "https://syncandshare.lrz.de/dl/fiWBtDLWz17RBc1Yd4VDW7/LLVM/default/18.1.8/Ubuntu/{{ ubuntu_version if ubuntu_version is defined else '20.04'}}/llvm.tar.xz"
  riscv_gcc.dl_url: "https://syncandshare.lrz.de/dl/fiWBtDLWz17RBc1Yd4VDW7/GCC/default/2024.09.03_gcc14/Ubuntu/{{ ubuntu_version if ubuntu_version is defined else '20.04'}}/rv32gc_ilp32d.tar.xz"
  riscv_gcc.dl_url_vext: "https://syncandshare.lrz.de/dl/fiWBtDLWz17RBc1Yd4VDW7/GCC/default/2024.09.03_gcc14/Ubuntu/{{ ubuntu_version if ubuntu_version is defined else '20.04'}}/rv32gcv_ilp32d.tar.xz"
flags: {}

Do not forget to set your `MLONMCU_HOME` environment variable first if not using the default location!

## Usage

The following examples demonstrates the powerful combination Postprocesses and Features provided by MLonMCU for the described use-case.

### A) Command Line Interface

Add the following arguments to your command:

`--feature log_instrs --config log_instrs.to_file=1 --postprocess analyse_instructions`

**Example:**

In [4]:
!mlonmcu flow run tinymlperf --backend tvmaotplus --target etiss --parallel --config mlif.num_threads=4 -c etiss.print_outputs=0 -c mlif.fuse_ld=lld \
        --config mlif.toolchain=gcc --config run.export_optional=1 --config mlif.strip_strings=1 \
        --post config2cols --config config2cols.limit="['mlif.optimize','mlif.lto']" \
        --post rename_cols --config rename_cols.mapping="{'config_mlif.optimize':'Optimize','config_mlif.lto':'LTO'}" \
        --post filter_cols --config filter_cols.keep="['Model','Simulated Instructions','ROM code','Optimize','LTO']" \
        --config-gen mlif.optimize=0 \
        --config-gen mlif.optimize=1 \
        --config-gen mlif.optimize=2 mlif.lto=0 --config-gen mlif.optimize=2 mlif.lto=1 \
        --config-gen mlif.optimize=3 mlif.lto=0 --config-gen mlif.optimize=3 mlif.lto=1 \
        --config-gen mlif.optimize=s mlif.lto=0 --config-gen mlif.optimize=s mlif.lto=1 \
        --config-gen mlif.optimize=z mlif.lto=0 --config-gen mlif.optimize=z mlif.lto=1 \
        --config-gen mlif.optimize=fast mlif.lto=0 --config-gen mlif.optimize=fast mlif.lto=1

INFO - Loading environment cache from file
INFO - Successfully initialized cache


INFO -  Processing stage LOAD


INFO -  Processing stage BUILD


ERROR - [16:25:01] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
[16:25:01] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
[16:25:01] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
Package "tflite.Model" is not installed. Hint: "pip install tlcpack[tvmc]".
ERROR - The process returned an non-zero exit code 5! (

ERROR - [session-0] [run-2] Run failed at stage 'BUILD', aborting...


...

ERROR - [16:25:24] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
[16:25:24] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
[16:25:24] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
Package "tflite.Model" is not installed. Hint: "pip install tlcpack[tvmc]".
ERROR - The process returned an non-zero exit code 5! (

ERROR - [16:25:24] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
[16:25:24] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
[16:25:24] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
Package "tflite.Model" is not installed. Hint: "pip install tlcpack[tvmc]".
ERROR - The process returned an non-zero exit code 5! (

ERROR - [16:25:24] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
[16:25:24] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
[16:25:24] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
Package "tflite.Model" is not installed. Hint: "pip install tlcpack[tvmc]".
ERROR - The process returned an non-zero exit code 5! (

INFO - Postprocessing session report


INFO - Done processing runs
INFO - Report:
     Model Optimize    LTO
0      aww        0  False
1      aww        1  False
2      aww        2      0
3      aww        2      1
4      aww        3      0
5      aww        3      1
6      aww        s      0
7      aww        s      1
8      aww        z      0
9      aww        z      1
10     aww     fast      0
11     aww     fast      1
12     vww        0  False
13     vww        1  False
14     vww        2      0
15     vww        2      1
16     vww        3      0
17     vww        3      1
18     vww        s      0
19     vww        s      1
20     vww        z      0
21     vww        z      1
22     vww     fast      0
23     vww     fast      1
24  resnet        0  False
25  resnet        1  False
26  resnet        2      0
27  resnet        2      1
28  resnet        3      0
29  resnet        3      1
30  resnet        s      0
31  resnet        s      1
32  resnet        z      0
33  resnet        z      1
34  resnet  

The interesting data is exported as an artifact and not shown in the report above. Therefore let's look at the generated files instead.

In [5]:
!mlonmcu export /tmp/exported/ -f --run
!ls /tmp/exported/

INFO - Loading environment cache from file
INFO - Successfully initialized cache
Creating directory: /tmp/exported
Done


load_metrics.csv  mlif	model_info.yml	report.csv  run.txt  toycar.tflite


CSV File of the used instruction classes (RISC-V only):

In [6]:
!cat /tmp/exported/analyse_instructions_majors.csv | column -t -s,

cat: /tmp/exported/analyse_instructions_majors.csv: No such file or directory


Frequently used instructions:

In [7]:
!cat /tmp/exported/analyse_instructions_seq1.csv | column -t -s,

cat: /tmp/exported/analyse_instructions_seq1.csv: No such file or directory


### B) Python Scripting

Python Imports

In [8]:
from tempfile import TemporaryDirectory
from pathlib import Path
import pandas as pd

from mlonmcu.context.context import MlonMcuContext
from mlonmcu.session.run import RunStage

Benchmark Configuration

In [9]:
FRONTEND = "tflite"
MODEL = "sine_model"
BACKEND = "tvmaotplus"
PLATFORM = "mlif"
TARGET = "etiss"
FEATURES = ["log_instrs"]
CONFIG = {"log_instrs.to_file": True}
POSTPROCESSES = ["analyse_instructions"]

Initialize and run a single benchmark

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [10]:
with MlonMcuContext() as context:
    with context.create_session() as session:
        run = session.create_run(config=CONFIG)
        run.add_features_by_name(FEATURES, context=context)
        run.add_frontend_by_name(FRONTEND, context=context)
        run.add_model_by_name(MODEL, context=context)
        run.add_backend_by_name(BACKEND, context=context)
        run.add_platform_by_name(PLATFORM, context=context)
        run.add_target_by_name(TARGET, context=context)
        run.add_postprocesses_by_name(POSTPROCESSES)
        session.process_runs(context=context)
        report = session.get_reports()
assert not session.failing
report.df

INFO - Loading environment cache from file


INFO - Successfully initialized cache


INFO - [session-1] Processing all stages


ERROR - [16:25:29] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
[16:25:29] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
[16:25:29] /tmp/CompareRISCVToolchains-de28/workspace/deps/src/tvm/src/target/parsers/aprofile.cc:97: Warning: Cannot parse target features for target: {"kind": "llvm", "mtriple": "arm64-apple-macos", "mcpu": "apple-latest"}. LLVM was not compiled with support for Arm(R)-based targets.
Package "tflite.Model" is not installed. Hint: "pip install tlcpack[tvmc]".


ERROR - The process returned an non-zero exit code 5! (CMD: `/tmp/CompareRISCVToolchains-de28/venv/bin/python3 -m tvm.driver.tvmc compile /tmp/CompareRISCVToolchains-de28/workspace/temp/sessions/1/runs/0/sine_model.tflite --target c --target-c-mcpu generic-rv32 --target-c-model etiss-rv32gc --runtime crt --executor aot --pass-config tir.disable_vectorize=True --pass-config tir.usmp.enable=True --pass-config tir.usmp.algorithm=hill_climb --input-shapes "dense_2_input:[1,1]" --opt-level 3 --output /tmp/tmpli0nidqz/default.tar -f mlf --model-format tflite --runtime-crt-system-lib 0 --executor-aot-unpacked-api 1 --executor-aot-interface-api c --target-c-constants-byte-alignment 16 --target-c-workspace-byte-alignment 16`)
Traceback (most recent call last):
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/session/run.py", line 1090, in process
    func()
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/session/run.py", line 942, in build
    _build()
  File "/home/runner/work/mlonmcu/mlonm

ERROR - [session-1] [run-0] Run failed at stage 'BUILD', aborting...


ERROR - All runs have failed to complete!


INFO - Postprocessing session report


INFO - [session-1] Done processing runs


AssertionError: 

The report is available as pandas dataframe.

Let's extract the artifacts created by the `analyse_instructions` postprocess:

In [ ]:
artifacts = session.runs[0].artifacts_per_stage[RunStage.POSTPROCESS]["default"]

**Visualization of used Instruction types**

In [ ]:
with TemporaryDirectory() as tmpdirname:
    dest = Path(tmpdirname) / "analyse_instructions_majors.csv"
    artifacts[0].export(dest)
    majors_df = pd.read_csv(dest)
    majors_df = majors_df.drop("Probablity", axis=1)
    majors_df.plot.bar(x="Major", title=f"Top Major Opcodes")

**Most used (groups of) intructions in the program**

In [ ]:
with TemporaryDirectory() as tmpdirname:
    for i in range(1, 4):
        dest = Path(tmpdirname) / f"analyse_instructions_seq{i}.csv"
        artifacts[i].export(dest)
        seq_df = pd.read_csv(dest)
        top = len(seq_df)
        seq_df = seq_df.drop("Probablity", axis=1)
        seq_df.plot.bar(x="Sequence", title=f"Top {top} {i}-length sequences")